In [1]:
import torch

In [3]:
# tensor ejemplo: tensor random con 3 valores
x = torch.randn(3)
x

tensor([-0.9073,  0.5067, -0.5938])

In [5]:
#calcular el gradiente de una función respecto a x debemos especificar el argumento requieres_grad=true. Por default es false
# cuando el requires_grad es falso no se calculan los gradients y lanzará error
# creando un tensor random con seguimiento de gradientes
x = torch.randn(3, requires_grad=True)
x

tensor([-0.3737, -0.9626,  0.7215], requires_grad=True)

In [7]:
# una operación con mi tensor ==> y=x+2
# inputs son x y 2, operacion +(nodo) y el output y
# se observa el atributo grad_fn por llamar el requieres_grad=true
# se apunta al AddBackward que es con lo que se buscan los backward pass
y = x + 2
y

tensor([1.6263, 1.0374, 2.7215], grad_fn=<AddBackward0>)

In [13]:
# esta función tambien tiene el atributo grad_fn y el MultBackward pues la operación es multiplicación
z = y * y * 2
z

tensor([ 5.2897,  2.1522, 14.8132], grad_fn=<MulBackward0>)

## Gradiente con un escalar(solo un valor, ej:mean)

In [9]:
# grad_fn es MeanBackward
z = z.mean()
z

tensor(7.4184, grad_fn=<MeanBackward0>)

In [10]:
# para calcular los gradientes hay que llamar z.backward()=(dz/dx). Calcula los gradientes de z con respecto a x
# como z es un valor escalar(un solo elemento) no tenemos que poner un argumento en las() de la función backward.
# la z solo tendrá un valor., lo que está bien para buscar el gradiente, no se multiplica por vectores o mas de un valor.
z.backward()
# x tiene un atributo .grad para calcular los gradientes de x
x.grad

tensor([2.1684, 1.3831, 3.6287])

## Gradiente con output no escalar (mas de un valor, ej un tensor o vector)

- Si el backward no es un solo valor(diferente al del caso de z.mean)

In [11]:
# valores o vector (1,3). Para llamar la backward function saldrá un error porque la funcion solo trabaja con valores escalares
z = y * y * 2
z

tensor([ 5.2897,  2.1522, 14.8132], grad_fn=<MulBackward0>)

In [12]:
z.backward()

RuntimeError: grad can be implicitly created only for scalar outputs

Si un Tensor no es escalar (más de 1 elemento) necesitamos especificar argumentos para backward()
Toca especificar un argumento de gradiente que es un tensor de forma coincidente o de igual tamaño (1,3 en este caso).
Necesario para el producto jacobiano vectorial

In [14]:
# Para corregir esto se crea un vector del mismo tamaño del output.
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float32) # producto de matrices
z.backward(v)
x.grad

tensor([2.8189, 5.5326, 3.6298])

### Prevenir que Pytorch trackee (siga) la historia calculando el atributo grad_fn
Por ejemplo cuando en un entrenamiento queremos actualizar weights y que no se sigan los gradientes.
Hay 3 opciones para que pytorch no siga nuestra historia de funciones gradiente:
- **x.requires_grad_(False)**
- **x.detach()**
- **Envolver función con 'with torch.no_grad():'**

In [16]:
#forma 1 de hacerlo
x = torch.randn(3, requires_grad=True)
x.requires_grad_(False) # siempre que la funcion tenga un _ al final en pytorch modificará el inplace de la variable
x # al llamar no tiene el requieres_grad lo que significa que no se estan siguiendo los gradientes

tensor([ 0.5751,  0.8912, -0.0337])

In [17]:
#forma 2 de hacerlo
x = torch.randn(3, requires_grad=True)
y = x.detach() #esto crea un tensor con los mismos valores pero sin requerir los gradientes
y

tensor([-0.0829,  1.6299, -0.1252])

In [18]:
y = x.detach() #esto crea un tensor con los mismos valores pero sin requerir los gradientes
y

tensor([-0.0829,  1.6299, -0.1252])

In [21]:
# forma 3
with torch.no_grad():
    y = x + 2
    print(y)

tensor([1.9171, 3.6299, 1.8748])


### Cuando se llama una funcion Backward los gradientes de un tensor se acumulan en el atributo .grad, asi que los valores se iran sumando.


In [22]:
# Un loop de entrenamiento de 3 iteraciones para evidenciar la acumulacion o sumatoria de gradientes
# esto evidencia que los gradientes pueden ser incorrectos con las iteraciones al sumarse
weights = torch.ones(4, requires_grad=True)
for epoch in range(3):
    # el ejemplo de una operacion de simulación
    model_output = (weights*3).sum()
    model_output.backward()
    print(weights.grad)

tensor([3., 3., 3., 3.])
tensor([6., 6., 6., 6.])
tensor([9., 9., 9., 9.])


In [24]:
#Forma correcta de establecer los gradientes en los pasos de entrenamiento
weights = torch.ones(4, requires_grad=True)
for epoch in range(3):
    # el ejemplo de una operacion de simulacion. 
    model_output = (weights*3).sum()
    model_output.backward()
    print(weights.grad)
    # funcion zero
    weights.grad.zero_()

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
